In [7]:
import aocd
data = aocd.get_data(day=1, year=2023)
T = data.split()

trans = {'one':'o1e', 'two':'t2o', 'three':'3e', 'four':'4',
         'five':'5e', 'six':'6', 'seven':'7n', 'eight':'e8t',
         'nine':'n9e'}

def tr(x):
    for s in trans: x = x.replace(s, trans[s])
    return x

def cal(x):
    l = [int(c) for c in x if c.isdigit()]
    return 10*l[0]+l[-1]

print("Part1:", sum(map(cal, T)), "Part2:",sum(map(cal, map(tr,T))))

Part1: 54388 Part2: 53515


In [23]:
import aocd
from collections import defaultdict
data = aocd.get_data(day=2, year=2023)

part1 = part2 = 0

for i, g in enumerate(data.split("\n")):
    _,l = g.split(": ")
    p1 = i+1
    maxes = defaultdict(int)
    for u in l.split("; "):
        for t in u.split(", "):
            n, c = t.split(" ")
            n = int(n)
            if not((c == "red" and n <= 12) 
                or (c == "green" and n <= 13) 
                or (c == "blue" and n <= 14)): p1 = 0
            maxes[c] = max(maxes[c], n)
    part1 += p1
    p2 = 1
    for k in maxes: p2 *= maxes[k]
    part2 += p2
print(part1,part2)

2795 75561


In [54]:
import aocd
data = aocd.get_data(day=3, year=2023)

mp = defaultdict(str)
T = data.split('\n')
n = len(T)
m = len(T[0])

for i,l in enumerate(T):
    for j,c in enumerate(l):
        if c != '.': 
            mp[(i,j)] = c

part1 = part2 = 0
gears = defaultdict(list)
tmp, ispart, cgear = 0, False, set()

for i in range(n):
    for j in range(m):
        if mp[(i,j)].isdigit():
            tmp = tmp*10+int(mp[(i,j)])
            for k in range(-1,2):
                for l in range(-1,2):
                    if mp[(i+k,j+l)] != '' and not(mp[(i+k,j+l)].isdigit()):
                        ispart = True
                    if mp[(i+k,j+l)] == '*': cgear.add((i+k,j+l))           
        else:
            if ispart:
                part1 += tmp
                for g in cgear: gears[g].append(tmp)
            tmp, ispart, cgear = 0, False, set()
    if ispart: 
        part1 += tmp
        for g in cgear: gears[g].append(tmp)
    tmp, ispart, cgear = 0, False, set()

for g in gears:
    if len(gears[g]) == 2:
        part2 += gears[g][0]*gears[g][1]
print("part1:",part1,"part2:",part2)

part1: 535351 part2: 87287096


In [32]:
import aocd
data = aocd.get_data(day=4, year=2023).splitlines()

part1, part2, n, cards = 0, 0, len(data), [1]*len(data)

for i, row in enumerate(data):
    win, hand = row.split(':')[1].split("|")
    match = len(set(win.split()).intersection(hand.split()))
    if match >= 1: part1 += 2**(match-1)
    for j in range(i+1, min(i+1+match, n)): cards[j] += cards[i]
    part2 += cards[i]

print("part1:", part1, "part2:", part2)

part1: 18653 part2: 5921508


In [46]:
import aocd
data = aocd.get_data(day=5, year=2023).split("\n\n")

seeds = list(map(int,data[0].split()[1:]))
part1 = [(seed,seed+1) for seed in seeds]
part2 = [(seeds[2*i],seeds[2*i]+seeds[2*i+1])
         for i in range(len(seeds)//2)]

def desc2trans(desc): return [ (src,src+length,dest-src)
           for line in desc.split('\n')[1:]
           for dest,src,length in [map(int,line.split())]]

def step(s2,transitions):
    ret = []
    for sta,sto in s2:
        for start, stop, delta in transitions:
            if start <= sta < stop:
                ret.append((sta+delta,min(sto,stop)+delta))
                sta, sto = min(sto,stop), sto
        if sta<sto: ret.append((sta,sto))
    return ret

for desc in data[1:]:
    transitions = sorted(desc2trans(desc))
    part1 = step(part1,transitions)
    part2 = step(part2,transitions)

print("part1:",min(part1)[0],"part2:",min(part2)[0])

part1: 199602917 part2: 2254686


In [108]:
import aocd, math, time

data = aocd.get_data(day=6, year=2023).split('\n')
races = list(zip(*map(lambda line: list(map(int,line.split()[1:])), data)))
t,d = map(lambda line: int(line.split(":")[1].replace(" ","")), data)

def win(t,d): return(t-1-int((t-math.sqrt(t*t -4*d))/2)*2)

def win_bf(t,d): return sum(1 for ti in range(t) if ti*(t-ti)>d )

print("Root calculation")
start_time = time.time()
print("part1:", math.prod(win(t,d) for t,d in races), "part2:", win(t,d))
print("--- %s seconds ---" % ((time.time() - start_time)))

print("Brute Force")
start_time = time.time()
print("part1:", math.prod(win_bf(t,d) for t,d in races), "part2:", win_bf(t,d))
print("--- %s seconds ---" % ((time.time() - start_time)))


Root calculation
part1: 2756160 part2: 34788142
--- 0.0010051727294921875 seconds ---
Brute Force
part1: 2756160 part2: 34788142
--- 72.09413647651672 seconds ---


In [44]:
import aocd
from collections import Counter

data = aocd.get_data(day=7, year=2023).split('\n')

def parse(s, part):
    hand, bid = s.split()
    bid,cnt = int(bid), Counter(hand)
    cards = ['23456789TJQKA', 'J23456789TQKA']
    
    if part == 2: js, cnt['J'] = cnt['J'], 0
    else: js = 0

    v = list(sorted(cnt.values(), reverse=True))
    v[0] += js
    return (v,tuple(map(cards[part-1].index,hand)),bid)

for part in [1,2]:
    T = sorted(map(lambda l:parse(l,part),data))
    res = sum( i*h[2] for i,h in enumerate(T[::],start=1) )
    print("Part"+str(part)+":",res)

Part1: 251058093
Part2: 249781879
